# **Contexte et but du code**

Nous avons deux codes pour tracer des abaques afin d'avoir une idée des paramètres L,D et H optimaux (voir schémas des codes des abaques). Cependant le code dépend beaucoup du choix des paramètres initiaux (initial guess) qui permettent de résoudre l'équation différentiel associé à notre problème de courbe élastique. Tout les paramètres ont été choisis pour des h=0.160 et R=2 cependant rien ne garantit que ces paramètres seront justes pour d'autres valeurs de h et R. Ce code est là pour nous donner une idée (un ordre de grandeur) de la valeurs des paramètres initiaux à mettre pour soit des paramètres L, D et H

# **Bibliothèques**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
import itertools

# **Paramètres (à modifier)**

In [2]:
D = 10  # mm
H = 5    # mm
L = 50  # mm
R = 2      # mm
h = 0.160  # mm
l = 40     # mm
alpha_0 = 0 # rad
alpha_f = -np.pi/2 #rad
GUESS = [0.3, -0.0005, 0.0001] # fair varier les paramètres (surtout GUESS[0])
# Si alpha_f > 0 alors GUESS[0] < 0 pour convergence 
# Sinon GUESS[0] > 0 pour convegence 
GUESS = [ 9.56365387e-02 -1.06132571e-04  3.96473480e-05]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1571138654.py, line 12)

# **Fonctions pour I(s)**

In [ ]:
def compute_I_interp(L):
    s = np.linspace(0, L, 200)
    b = np.zeros_like(s)
    for i in range(len(s)):
        if s[i] < R:
            y_circ = np.sqrt(R**2 - (s[i] - R)**2)
            b[i] = l + 2 * (R - y_circ)
        elif s[i] > s[-1] - R:
            y_circ = np.sqrt(R**2 - ((s[-1] - R) - s[i])**2)
            b[i] = l + 2 * (R - y_circ)
        else:
            b[i] = l
    I = (b * h**3) / 12
    dI_ds = np.gradient(I, s)
    return interp1d(s, I, kind='cubic', fill_value='extrapolate'), interp1d(s, dI_ds, kind='cubic', fill_value='extrapolate')

# **ODE de l’élastique**

In [ ]:
def elastica_odes(s_val, Y, lambda_x, lambda_y, I_func, dI_ds_func):
    theta, kappa, x, y = Y
    I = I_func(s_val)
    dI = dI_ds_func(s_val)
    return [kappa,
        (lambda_x * np.sin(theta) - lambda_y * np.cos(theta) - dI * kappa) / I,
        np.cos(theta),
        np.sin(theta)]

# **Résolution via méthode de tir avec exploration automatique**

In [ ]:
def solve_elastica(L, D, H, initial_guess, n_try=5, perturb=0.05):
    I_func, dI_ds_func = compute_I_interp(L)

    def shooting(params):
        kappa0, lambda_x, lambda_y = params
        Y0 = [0, kappa0, 0, 0]
        sol = solve_ivp(elastica_odes, (0, L), Y0, args=(lambda_x, lambda_y, I_func, dI_ds_func), dense_output=True)
        theta_f, x_f, y_f = sol.y[0, -1], sol.y[2, -1], sol.y[3, -1]
        return [
            (theta_f - alpha_f) / np.pi,
            (x_f - D) / (abs(D) + 1e-8),
            (y_f - H) / (abs(H) + 1e-8)
        ]

    # Tentative principale
    sol = fsolve(shooting, initial_guess, xtol=1e-9, factor=0.1, full_output=True)
    opt_params, _, ier, _ = sol

    # Si échec, on explore autour de initial_guess
    if ier != 1:
        print("Convergence échouée, recherche autour de l'initial_guess...")
        best_params = None
        best_residual = np.inf

        # Crée des perturbations autour de initial_guess
        perturbations = [np.linspace(val*(1-perturb), val*(1+perturb), n_try) for val in initial_guess]
        for guess in itertools.product(*perturbations):
            try:
                sol_try = fsolve(shooting, guess, xtol=1e-9, factor=0.1, full_output=True)
                params, _, ier_try, _ = sol_try
                if ier_try == 1:
                    res = np.linalg.norm(shooting(params))
                    if res < best_residual:
                        best_residual = res
                        best_params = params
            except:
                continue

        if best_params is None:
            raise RuntimeError(f"Échec convergence après recherche autour de initial_guess pour L = {L}")
        else:
            opt_params = best_params
            print(f"Convergence trouvée après exploration : kappa0={opt_params[0]:.6f}, lambda_x={opt_params[1]:.6e}, lambda_y={opt_params[2]:.6e}")

    # Résolution finale avec opt_params
    kappa0, lambda_x, lambda_y = opt_params
    Y0 = [alpha_0, kappa0, 0, 0]
    sol_ivp = solve_ivp(elastica_odes, (0, L), Y0, args=(lambda_x, lambda_y, I_func, dI_ds_func), dense_output=True)
    s = np.linspace(0, L, 400)
    x, y = sol_ivp.sol(s)[2], sol_ivp.sol(s)[3]
    return x, y, opt_params


# **Résolution pour L**

In [ ]:
x1, y1, opt1 = solve_elastica(L, D, H, initial_guess=GUESS)
print("Paramètres optimaux :", opt1)

# **Affichage**

In [ ]:
plt.figure(figsize=(5,5), dpi=300)
plt.plot(x1, y1, label=f'L = {L:.2f} mm', linewidth=2, color='k')
plt.scatter([0, D], [0, H], color='red', label='Points d’ancrage', zorder=5)
plt.title(f'Courbes élastiques (D = {D} mm, H = {H} mm)')
plt.xlabel('x (mm)')
plt.ylabel('y (mm)')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()